In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset

# For reproducibility
np.random.seed(42)
torch.manual_seed(42) # 乱数生成シード
cudnn.benchmark = True

# Grab a GPU if there is one
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using {} device: {}".format(device, torch.cuda.current_device()))
else:
    device = torch.device("cpu")
    print("Using {}".format(device))

Using cuda device: 0


In [26]:
fold = "../../train_raw_npy/"
acc_xyz = np.load(f"{fold}acc_xyz.npy")
label = np.load(f"{fold}label.npy")
label = label.reshape(-1,1,500)
print(acc_xyz.shape)
print(label.shape)
xyz_label = np.concatenate([acc_xyz, label], axis=1)
print(xyz_label.shape)
print(xyz_label)

(196072, 3, 500)
(196072, 1, 500)
(196072, 4, 500)
[[[  3.859912   3.864048   3.897538 ...  -5.213781  -4.404522  -4.079691]
  [  1.117648   0.986799   0.96659  ...  -2.146069  -2.34677   -2.528258]
  [  8.659125   8.763011   8.724693 ...  -8.829224 -10.52848  -11.961449]
  [  4.         4.         4.       ...   4.         4.         4.      ]]

 [[ -3.881881  -3.832097  -4.384022 ...  -3.763638  -3.744682  -3.810714]
  [ -2.396385  -1.951583  -1.114181 ...  -1.695331  -1.790109  -1.838136]
  [-12.083903 -11.001618 -10.145183 ...  -9.403623  -9.081378  -9.258451]
  [  4.         4.         4.       ...   4.         4.         4.      ]]

 [[ -4.041412  -4.346396  -4.792192 ...   4.79784    4.74148    4.872849]
  [ -1.742979  -1.447526  -0.83161  ...  -0.029058  -0.160565  -0.257327]
  [ -9.749189 -10.292441 -10.408511 ... -10.046281 -10.13082  -10.538704]
  [  4.         4.         4.       ...   4.         4.         4.      ]]

 ...

 [[ -1.7261    -1.482551  -1.129425 ...  -1.69509

In [27]:
xyz_label_20000 = xyz_label[-30000:-10000,:,:]
xyz_label_20000 = xyz_label_20000.transpose(0,2,1).reshape(-1,4)
print(xyz_label_20000.shape)
print(xyz_label_20000)

(10000000, 4)
[[ 4.078755  6.114164 -8.679988  4.      ]
 [ 4.484264  3.687497 -7.649245  4.      ]
 [ 4.940645  3.558285 -7.373154  4.      ]
 ...
 [-9.378911 -0.699875 -3.37502   6.      ]
 [-9.504357 -0.83574  -3.453182  6.      ]
 [-9.749665 -0.80149  -3.371619  6.      ]]


In [28]:
df = pd.DataFrame(xyz_label_20000, columns=["acc_x", "acc_y", "acc_z", "label"])

df["label"] = df["label"].astype(int).astype(str)

df.to_csv(f"{fold}first.csv", index=False)

In [29]:
# データを10個のCSVファイルに分割して保存
num_files = 10
rows_per_file = 1000000

for i in range(num_files):
    start_row = i * rows_per_file
    end_row = start_row + rows_per_file
    subset = xyz_label_20000[start_row:end_row]
    
    df = pd.DataFrame(subset, columns=["acc_x", "acc_y", "acc_z", "label"])
    
    # sbj_id列を追加
    df["sbj_id"] = i
    
    # label列を文字列に変換
    df["label"] = df["label"].astype(int).astype(str)
    
    # 列の順序を変更
    df = df[["sbj_id", "acc_x", "acc_y", "acc_z", "label"]]
    
    # CSVファイルに保存
    csv_file_path = f"sbj_{i}.csv"
    df.to_csv(csv_file_path, index=False)

    print(f"データが{csv_file_path}に保存されました。")

データがsbj_0.csvに保存されました。
データがsbj_1.csvに保存されました。
データがsbj_2.csvに保存されました。
データがsbj_3.csvに保存されました。
データがsbj_4.csvに保存されました。
データがsbj_5.csvに保存されました。
データがsbj_6.csvに保存されました。
データがsbj_7.csvに保存されました。
データがsbj_8.csvに保存されました。
データがsbj_9.csvに保存されました。
